In [2]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import re
import openpyxl

In [3]:
#cargar primero el excel o csv
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    nombre_col = ['n_afi', 'parentesco','nombre','dni','no_se','dni2','plan','plan2','sexo','localidad','provincia']
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron_capita= pd.read_csv(ruta_archivo, encoding='ISO-8859-1', sep=';', names=nombre_col)
            #padron_capita = pd.read_excel(ruta_archivo)
            print(padron_capita)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/odontologia (1).csv' mode='r' encoding='cp1252'>
              n_afi  parentesco                          nombre       dni  \
0     1002682185800           0  LUGO RUBEN OMAR                 19780915   
1     1003766928700           0  VELAZQUEZ MAXIMILIANO LEONEL    19930208   
2     1001712668200           0  CAMPOS Sandra Raquel            19640424   
3     1002399314500           0  AHUMADA ALBERTO ORLANDO         19740809   
4     1003281958600           0  PELAYES LEONARDO DAMIAN         19860912   
...             ...         ...                             ...       ...   
5041   403451270601           1  ROJAS ROCIO CECILIA             19971105   
5042   403451270602           2  DE LIMA ULISES ROMAN            20170810   
5043   403451270603           2  DE LIMA LAUTARO ANDRES          20140109   
5044   403698885902           2  FEGO GIANLUCA BENJAMIN          20211207   
5045   403840299602           2  LOPEZ FRUTO MELODY SO

In [15]:
diccionario ={'0':'titular',
              '1':'familiar',
              '2':'familiar',
              '3':'familiar',
              '4':'familiar',
              '5':'familiar',
              '6':'familiar',
              '7':'familiar',
              '8':'familiar'
              }      
#padron_capita['parentesco']=padron_capita['parentesco'].str.replace('\d|\(|\)', '', regex=True)
padron_capita['parentesco']=padron_capita['parentesco'].astype(str).replace(diccionario)

In [22]:
sedes={
'010': 'CABA',
'020': 'ZONA NORTE Y NOROESTE',
'030': 'ZONA OESTE',
'040': 'ZONA SUR',
'050': 'LA PLATA',
'060': 'MAR DEL PLATA',
'070': 'PERGAMINO',
'080': 'SANTA FE',
'090': 'CORDOBA',
'100':'MENDOZA',
'110':'TUCUMAN'
}
padron_capita['n_afi']=padron_capita['n_afi'].astype(str).str.zfill(13)
padron_capita['sede']=padron_capita['n_afi'].str[:3].replace(sedes)



In [12]:
save_path = filedialog.askdirectory()
padron_capita.to_excel(save_path + '\capitasagosto.xlsx', engine='openpyxl', index=False)